In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
from Helper import * 
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from IPython.display import Image,display
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
from collections import OrderedDict

In [ ]:
namespace1="broad-genomics-delivery"
workspace1="Getz_IBM_CellLines_RNASeqData"
namespace2="broad-firecloud-ccle"
workspace2="CCLE_DepMap_RNAseq"
namespace3="broad-genomics-delivery"
workspace3="CCLE_DepMap_RNAseq"
namespace4="broad-genomics-delivery"
workspace4="Cancer_Cell_Line_Factory_CCLF_RNAseq"
namespace5="nci-mimoun-bi-org"
workspace5="CCLF_RNA_2_0"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_hg38_RNAseq"
source1="ibm"
source2="ccle"
source3="ccle"
source4="cclf"
samplesetname="19Q3"
release = samplesetname

In [ ]:
wm1 = dm.WorkspaceManager(namespace1, workspace1)
wm2 = dm.WorkspaceManager(namespace2, workspace2)
wm3 = dm.WorkspaceManager(namespace3, workspace3)
wm4 = dm.WorkspaceManager(namespace4, workspace4)

refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [ ]:
## made some changes; now most all TerraFunction functions take a string of "namespace/workspace" 
## instead of the workspace manager object
## addToSampleSet, addToPairSet, updateAllSampleSet, waitForSubmission...

name_work1 = namespace1 + '/' + workspace1
name_work2 = namespace2 + '/' + workspace2
name_work3 = namespace3 + '/' + workspace3
name_work4 = namespace4 + '/' + workspace4
refname_work = refnamespace + '/' + refworkspace

# Generate sample set from new samples

In [ ]:
newsamples = createDatasetWithNewCellLines(refwm, samplesetname, wm1, source1,
wm2, source2,
wm3, source3,
wm4, source4)

# run the pipeline

In [ ]:
samtofastq = refwm.get_config("samtofastq_v1-0_BETA_cfg")
samtofastq

In [ ]:
samtofastq['samtofastq_workflow.samtofastq.input_bam_cram']= 'this.WES_bam'
refwm.update_config(samtofastq)
submission_id = refwm.create_submission(samtofastq['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refname_work, submission_id)

In [ ]:
star = refwm.get_config("star_v1-0_BETA_cfg")
star

In [ ]:
refwm.update_config(star)
submission_id = refwm.create_submission(star['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
rnasnps = refwm.get_config("rnaseq-germline-snps-indels")
rnasnps

In [ ]:
refwm.update_config(rnasnps)
submission_id_snp = refwm.create_submission(rnasnps['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refname_work, submission_id)

In [ ]:
rsem = refwm.get_config("rsem_v1-0_BETA_cfg")
rsem

In [ ]:
refwm.update_config(rsem)
submission_id1 = refwm.create_submission(rsem['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refname_work, submission_id1)

In [ ]:
fusion = refwm.get_config("hg38_STAR_fusion")
fusionw

In [ ]:
fusion['inputs']['trinity_cleaned.samtofastq.input_bam'] = 'this.WES_bam'
refwm.update_config(fusion)
submission_id2 = refwm.create_submission(fusion['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
aggregate = refwm.get_config("rsem_aggregate_results_v1-0_BETA_cfg")
aggregate

In [ ]:
updateAllSampleSet('CCLE_'+samplesetname, Allsample_setname='DM19Q2_PATHS_CORRECTED_V2')

In [ ]:
submission_id1 = refwm.create_submission(aggregate['name'], 'All_samples')

## Expression post processing

In [ ]:
terra.wait_for_submission(refname_work, [submission_id1, submission_id_snp, submission_id2])

In [ ]:
%%R
release <- '19Q3'

In [ ]:
%%R
library('taigr')

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')

In [ ]:
%%R
source('CCLE_postp_function.R')

In [ ]:
res = refwm.get_sample_sets().loc['all_samples']
res

In [ ]:
res = refwm.get_sample_sets().loc['All_samples']
rsem_genes_expected_count = res['rsem_genes_expected_count']
rsem_genes_tpm = res['rsem_genes_tpm']
rsem_transcripts_tpm = res['rsem_transcripts_tpm']
! gsutil cp $rsem_genes_expected_count "temp/" & gsutil cp $rsem_genes_tpm "temp/" & gsutil cp $rsem_transcripts_tpm "temp/"

In [ ]:
%%R
# alternative one
download_paths <- list(
  counts_genes='temp/DM19Q2_PATHS_CORRECTED_V2.rsem_genes_expected_count.txt.gz',
  tpm_genes='temp/DM19Q2_PATHS_CORRECTED_V2.rsem_genes_tpm.txt.gz',
  tpm_transcripts='temp/DM19Q2_PATHS_CORRECTED_V2.rsem_transcripts_tpm.txt.gz')

In [ ]:
%%R
# These files are downloaded from FireCloud/Terra
download_paths <- list(
  tpm_genes='temp/expression.genes.tpm.txt.gz',
  tpm_transcripts='temp/expression.transcript.tpm.txt.gz',
  counts_genes='temp/expression.expectedcount.txt.gz')

In [ ]:
%%R
tpm_transcripts = readTranscripts(download_paths$tpm_transcripts)
counts_genes = readCounts(download_paths$counts_genes)
tpm_genes = readTPM(download_paths$tpm_genes)

In [ ]:
%%R
# Fix the colnames (for cases where there are mixed values (CCLE_name and DepMap_IDs))
colnames(counts_genes) %<>% renameFunction(.)
colnames(tpm_genes) %<>% renameFunction(.)
colnames(tpm_transcripts) %<>% renameFunction(.)

### data exploration and QC

In [ ]:
%%R
tpm_genes[,'ACH-001767']

In [ ]:
%%R 
# Quick QC
# We are looking for samples with a worrying amount of zeros
zero_threshold <- 39000
number_zeros <- apply(tpm_genes[,3:ncol(tpm_genes)], 2, FUN = function(x) length(x[x == 0]))
nzdf <- data.frame(CL=names(number_zeros), nz=number_zeros, stringsAsFactors = F)

In [ ]:
%%R
number_zeros <- number_zeros[order(-number_zeros)]
number_zeros <- number_zeros[number_zeros < zero_threshold]
pass <- number_zeros %>% names()

# These samples failed
failed <- setdiff(colnames(tpm_genes), pass) %>% .[!(. %in% c('gene_id', 'transcript_id(s)'))]

counts_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_transcripts %<>% dplyr::select(c("transcript_id", "gene_id", pass))

In [ ]:
%%R
failed

In [ ]:
%%R 

In [ ]:
%%R
# Plot of the samples that fail
plot <- ggplot(nzdf, aes(nz)) +
  geom_histogram(bins = 100, color='black', fill='white') +
  geom_vline(xintercept = zero_threshold, linetype=2) +
  geom_label_repel(data = nzdf %>% filter(nz > zero_threshold), aes(x=nz, y=0, label=CL), size=5, fill=rgb(1,1,1,0.5))

ggsave(plot, filename ='temp/ggplot.png', width=20, height = 20)

In [ ]:
display(Image(filename='temp/ggplot.png'))

In [ ]:
%%R
write.table(
  counts_genes, 
  file = paste0('temp/expression.', release,'.counts.tsv'), 
  sep = '\t', row.names = F, quote = F)
write.table(
  tpm_genes, 
  file = paste0('temp/expression.', release,'genes.tsv'), 
  sep = '\t', row.names = F, quote = F)
write.table(
  tpm_transcripts, 
  file = paste0('temp/expression.', release,'.transcripts.tsv'),
  sep = '\t', row.names = F, quote = F)

# Validation

In [ ]:
counts_genes = pd.read_csv('temp/expression.'+ release + '.counts.tsv', sep='\t')

In [ ]:
tpm_genes = pd.read_csv('temp/expression.'+ release + '.tpm.tsv', sep='\t')
tpm_transcripts = pd.read_csv('temp/expression.'+ release + '.transcripts.tsv', sep='\t')

In [ ]:
DM19Q2counts = tc.get(name='depmap-expression-87f8', version=7, file='DM19Q2.counts')
DM19Q2tpm = tc.get(name='depmap-expression-87f8', version=7, file='DM19Q2.tpm')
DM19Q2transcripts = tc.get(name='depmap-expression-87f8', version=7, file='DM19Q2.transcripts')

In [ ]:
rna_ibm_embargo = [i for i in gsheets['RNAseq_IBM_embargo'].values.tolist() if i is not np.nan]
rna_dmc_embargo = [i for i in gsheets['RNAseq_DMC_embargo'].values.tolist() if i is not np.nan]

In [ ]:
tpm_genes['ACH-000309'] = DM19Q2tpm['ACH-000309']
tpm_genes['ACH-001852'] = DM19Q2tpm['ACH-001852']
tpm_transcripts['ACH-001852'] = DM19Q2transcripts['ACH-001852']
tpm_transcripts['ACH-000309'] = DM19Q2transcripts['ACH-000309']
counts_genes['ACH-000309'] = DM19Q2counts['ACH-000309']
counts_genes['ACH-001852'] = DM19Q2counts['ACH-001852']

In [ ]:
tpm_genes.to_csv('temp/expression.'+ release + '.genes.tsv', sep='\t')
tpm_transcripts.to_csv('temp/expression.'+ release + '.transcripts.tsv', sep='\t')
counts_genes.to_csv('temp/expression.'+ release + '.counts.tsv', sep='\t')

In [ ]:
counts_genes.columns.str.contains('ibm')

In [ ]:
prev = tc.get(name='depmap-expression-87f8', version=7, file='DM19Q2.tpm')

In [ ]:
tpm_genes[tpm_genes.columns[2:]] = tpm_genes[tpm_genes.columns[2:]].apply(lambda x: np.log2(x+1))

In [ ]:
new1 = set(counts_genes.columns.values.tolist())
prev = set(prev.columns.values.tolist())

In [ ]:
new3 = set(tpm_transcripts.columns.values.tolist())
new2 = set(tpm_genes.columns.values.tolist())

In [ ]:
print(len(prev), len(prev & new2), len(new1), len(new1 & new2 & new3), new2-prev, prev-new2)

In [ ]:
metadata = tc.get(name='internal-19q2-9504', version=24, file='sample_info')

In [ ]:
metadata

In [ ]:
# finding train and test set
trainame = [val for val in new1&prev if val[:3] == 'ACH']
testname = [val for val in new1-prev if val[:3] == 'ACH']

#looking at the 2000 most variable genes in the two sets
genetolookfor = 2000
gene_var = counts_genes[trainame].var(1).values
print(len(gene_var))
sorting = np.argsort(gene_var)[-genetolookfor:]

In [ ]:
unregistered = set(testname) - set(metadata["DepMap_ID"].values.tolist())
unregistered

In [ ]:
len(counts_genes['ACH-001767']) - np.count_nonzero(counts_genes['ACH-001767'].values)

In [ ]:
# creating and reodering train and test sets
traindata = counts_genes[set(trainame)-unregistered].values[sorting].T
trainlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(trainame)-unregistered].columns.tolist() if val not in unregistered]

testdata = counts_genes[set(testname)-unregistered].values[sorting].T
testlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(testname)-unregistered].columns.tolist() if val not in unregistered]

In [ ]:
traindata.shape

In [ ]:
# learn KNN classifier to the metadata diseases
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(traindata, trainlabels) 
predicted = neigh.predict(testdata)

In [ ]:
predicted[1]

In [ ]:
labels = trainlabels + testlabels
colors=[0]*len(trainlabels)
colors.extend([1,2,2,2,2,1,2,2,2,1,2])
data = np.vstack([traindata,testdata])

In [ ]:
# plot them with TSNE, highlight the points that failed and show colors for diseases
dimred = TSNE(2,10).fit_transform(data)

In [ ]:
scatter(dimred, labels=labels,colors=colors, radi=1.9)

## Save files for taiga

In [ ]:
tc.update_dataset(dataset_permaname="depmap-expression-87f8",
                 upload_file_path_dict={'temp/expression.'+release+'.transcripts.tsv': 'TableTSV',
                                       'temp/expression.'+release+'.genes.tsv': 'TableTSV',
                                       'temp/expression.'+release+'.counts.tsv': 'TableTSV'},
                 dataset_description="# Updating to "+release+"""
                 adding missing samples in Terra merge files
                 
                 
                 """)

## Fusion post processing

In [ ]:
terra.wait_for_submission(submission_id2)

In [ ]:
aggregate = refwm.get_config('Aggregate_Fusion_Calls')
aggregate

In [ ]:
refwm.update_config(aggregate)
submission_id2 = refwm.create_submission(aggregate['name'], 'All_samples')

In [ ]:
terra.wait_for_submission(submission_id2)

In [ ]:
refwm.get_sample_sets().loc['All_samples']['fusions_star']

In [ ]:
! gsutil cp $aggregated "temp/expression.fusion.tsv"

### Overview

This document contains the code used to generate the unfiltered and filtered versions of the fusion datasets for the release. The bottom of the document also contains some comparisons between the release fusion dataset, CCLE2 fusion calls, and the translocation data from CCLE2.

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
source("CCLE_postp_function.R")
filepath <- 'temp/expression.fusion.tsv'
release <- '19Q3'

## Generate filtered fusion table

Release: `r release`

We want to apply filters to the fusion table to reduce the number of artifacts in the dataset. Specifically, we filter the following:

* Remove fusions involving mitochondrial chromosomes, or HLA genes, or immunoglobulin genes
* Remove red herring fusions (from STAR-Fusion annotations column)
* Remove recurrent in CCLE (>= 25 samples)
* Remove fusion with (SpliceType=" INCL_NON_REF_SPLICE" and LargeAnchorSupport="No" and FFPM < 0.1)
* Remove fusions with FFPM < 0.05 (STAR-Fusion suggests using 0.1, but looking at the translocation data, this looks like it might be too aggressive)

In [ ]:
%%R
unfiltered_fusions <- readFusions(filepath)
filtered_fusions <- filterFusions(unfiltered_fusions)

In [ ]:
%%R
# Save the files (to be uploaded to taiga)
write.table(
  unfiltered_fusions,
  file = paste0('temp/fusions.',release, '.unfiltered.tsv'),
  sep = '\t', quote = F, row.names = F
)
write.table(
  filtered_fusions,
  file = paste0('temp/fusions.', release, '.filtered.tsv'),
  sep = '\t', quote = F, row.names = F
)

# Validation

## Validation Protocol:

to validate fusions, one should be able to list all cells with known fusions (i.e. elwing sarcoma) and check for each new cell in this set of knownfusioncells, if the fusion is present or not. and validate the fusion quality this way.

In [ ]:
# check that all cells lines are present on fusion unfiltered
df = pd.read_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
new = set([i.split('_')[1][:10] for i in list(set(df["DepMap_ID"].tolist()))])
print(prev - new, new-prev)

In [ ]:
# remove fusions from the same samples as for that failed expression threshold


In [ ]:
unfiltered = pd.read_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
filtered = pd.read_csv('temp/fusions.'+ release+ '.filtered.tsv',sep='\t')

In [ ]:
unfiltered["DepMap_ID"] = ['ACH' + i.split('_ACH')[1].split('.')[0] for i in unfiltered["DepMap_ID"]]

In [ ]:
filtered["DepMap_ID"] = ['ACH' + i.split('_ACH')[1].split('.')[0] for i in filtered["DepMap_ID"]]

In [ ]:
q1filtered = tc.get(name='depmap-fusions-7990', version=1, file='fusions.19q1.filtered')
q1unfiltered = tc.get(name='depmap-fusions-7990', version=1, file='fusions.19q1.unfiltered')

In [ ]:
filtered

In [ ]:
q1filtered

In [ ]:
filtered = pd.concat([filtered,q1filtered[q1filtered['DepMap_ID']=='ACH-000309']], sort=False)
unfiltered = pd.concat([unfiltered,q1unfiltered[q1unfiltered['DepMap_ID']=='ACH-000309']], sort=False)

In [ ]:
unfiltered.to_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
filtered.to_csv('temp/fusions.'+ release+ '.filtered.tsv',sep='\t')

# Uploading to Taiga

In [ ]:
tc.update_dataset(dataset_permaname="depmap-fusions-7990",
                     upload_file_path_dict={'temp/fusions.'+release+'.filtered.tsv': 'TableTSV',
                                        'temp/fusions.'+release+'.unfiltered.tsv': 'TableTSV'},
                 dataset_description="# Updating to "+release+"""
                 
                 reparing the missing cell lines from Terra
                 """)

# IF want to merge here instead of on Terra:

In [ ]:
newsamples

In [ ]:
prevsamplesets = ['CCLE_19Q3interim',samplesetname]
samples = []
for i in prevsamplesets:
    samples.extend(refwm.get_sample_sets().loc[i].samples)
res = []
terrasamp = refwm.get_samples()
for i, sample in enumerate(samples):
    res.append(terrasamp.loc[sample])
    genes_fusion = res[i]['fusion_predictions_abridged']
    rsem_genes_transcripts = res[i]['rsem_isoforms']
    rsem_genes_expected_count = res[i]['rsem_genes']
    ! gsutil cp $rsem_genes_expected_count 'temp/' && gsutil cp $rsem_genes_transcripts 'temp/' && gsutil cp $genes_fusion 'temp/'

In [ ]:
mainres = refwm.get_sample_sets().loc['DM19Q2_PATHS_CORRECTED_V2']
maingenes_fusion = mainres['fusions_star']
mainrsem_genes_tpm = mainres['rsem_genes_tpm']
mainrsem_genes_transcripts = mainres['rsem_transcripts_tpm']
mainrsem_genes_expected_count = mainres['rsem_genes_expected_count']
! gsutil cp $mainrsem_genes_expected_count "temp/" && gsutil cp $mainrsem_genes_transcripts "temp/" && gsutil cp $maingenes_fusion "temp/expression.fusion.tsv" && gsutil cp $mainrsem_genes_tpm "temp/"

In [ ]:
mainres['rsem_genes_expected_count']

In [ ]:
addSamplesRSEMToMain(res,mainres)

In [ ]:
genes_fusion = ['temp/'+val['fusion_predictions_abridged'].split('/')[-1] for val in res]
addToMainFusion(genes_fusion,'temp/expression.fusion.tsv')